In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=1eabd757bfe8ff55fe1da69270e1084ebc744b2a1c7e69a4b9b750902f52299b
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.8MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F13 '''
def F13(X):
    f = bn.F13()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.07412088568349123 ,random_state=0 , l1_ratio=0.98989898989899, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 18.264432 , C = 956.144428 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_13_1000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_13_200Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [5]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,51):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 50, [5] * 50 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 50000)

n_obs , dim =  30, 50
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,150])
for i in range(G[1].shape[0]):
        min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
        min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
        min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
        X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=858064, Mon Apr 13 20:10:11 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 4.660812755445248e+03 1.0e+00 2.36e+00  2e+00  2e+00 0:00.0
    2     30 4.627795292700410e+03 1.0e+00 2.28e+00  2e+00  2e+00 0:00.0
    3     45 4.396369893660128e+03 1.1e+00 2.22e+00  2e+00  2e+00 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [4964.6594534] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [5175.88708399] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1500 1.443319829726692e+03 1.5e+00 2.60e-01  2e-01  3e-01 0:01.8
  200   3000 1.373848069978619e+03 1.5e+00 1.29e-02  1e-02  1e-02 0:03.5
  300   4500 1.373735597568421e+03 1.6e+00 8.36e-04  7e-04  8e-04 0:05.3
  400   6000 1.373735088653906e+03 1.6e+00 5.63e-05  5e-05  5e-05 0:07.0
  500   7500 1.373735086434118e+03 1.6e+00 3.18e-06  3e-06  3e-06 0:08.8
  600   9000 1.373735086429394e+03 1.6e+00 1.93e-07  1e-07  2e-07 0:10.5
  700  10500 1.373735086429367e+03 1.7e+00 2.84e-08  2e-08  2e-08 0:12.2
  726  10890 1.373735086429368e+03 1.7e+00 2.23e-08  2e-08  2e-08 0:12.7
termination on tolfun=1e-11 (Mon Apr 13 20:10:26 2020)
final/bestever f-value = 1.373735e+03 1.373735e+03
incumbent solution: [ 1.78280393 -0.85595337 -0.68104966  0.48160334 -2.20873819  0.42033646
 -1.02670657 -1.2844278  ...]
std deviations: [1.74103468e-08 1.66111861e-08 1.68239733e-08 1.69026124e-08
 1.76275768e-08 1.71545603e-08 1.70341173e-08 1.80545294e-08 ...]
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimen

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [5086.39851569] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=5086.398516, sigma=2.37e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=5086.398516, sigma=2.37e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')

  100   1500 2.375699848704098e+03 1.5e+00 5.97e-01  6e-01  6e-01 0:02.1
  200   3000 2.287571116222488e+03 1.5e+00 7.54e-02  6e-02  8e-02 0:04.2
  300   4500 2.285753800362386e+03 1.9e+00 1.29e-02  1e-02  1e-02 0:06.3
  400   6000 2.285680393496262e+03 2.4e+00 2.72e-03  2e-03  3e-03 0:08.4
  500   7500 2.285678992875557e+03 2.6e+00 2.80e-04  2e-04  3e-04 0:10.5
  600   9000 2.285678981766588e+03 2.8e+00 2.46e-05  2e-05  3e-05 0:12.6
  700  10500 2.285678981673353e+03 3.0e+00 1.83e-06  1e-06  2e-06 0:14.7
  800  12000 2.285678981672636e+03 3.3e+00 2.28e-07  1e-07  3e-07 0:16.8
  900  13500 2.285678981672624e+03 3.4e+00 5.74e-08  3e-08  6e-08 0:18.9
  913  13695 2.285678981672625e+03 3.4e+00 4.63e-08  2e-08  5e-08 0:19.2
termination on tolfun=1e-11 (Mon Apr 13 20:10:48 2020)
final/bestever f-value = 2.285679e+03 2.285679e+03
incumbent solution: [ 5.         -4.23348347 -0.8546936   1.48802417 -3.81946342 -0.28562671
 -0.58542412 -0.89747607 ...]
std deviations: [2.65134868e-08 3.6552881

...

  100   1500 2.314529331324991e+03 1.5e+00 3.67e-01  3e-01  4e-01 0:02.1
  200   3000 2.286085466505650e+03 1.6e+00 3.83e-02  3e-02  4e-02 0:04.2
  300   4500 2.285685082721497e+03 1.7e+00 5.31e-03  4e-03  6e-03 0:06.3
  400   6000 2.285679066471559e+03 2.0e+00 6.16e-04  4e-04  6e-04 0:08.3
  500   7500 2.285678983084800e+03 2.4e+00 7.93e-05  5e-05  8e-05 0:10.5
  600   9000 2.285678981684103e+03 2.7e+00 8.54e-06  5e-06  9e-06 0:12.6
  700  10500 2.285678981672727e+03 3.0e+00 8.65e-07  5e-07  9e-07 0:14.6
  800  12000 2.285678981672625e+03 3.3e+00 9.05e-08  5e-08  9e-08 0:16.8
  858  12870 2.285678981672623e+03 3.3e+00 4.79e-08  2e-08  5e-08 0:18.1
termination on tolfun=1e-11 (Mon Apr 13 20:28:37 2020)
final/bestever f-value = 2.285679e+03 2.285679e+03
incumbent solution: [ 5.         -4.23348324 -0.85469361  1.48802418 -3.81946336 -0.28562649
 -0.58542438 -0.89747613 ...]
std deviations: [2.72593943e-08 3.66077226e-08 3.73255580e-08 4.34502077e-08
 3.25031352e-08 4.02088258e-08 3.7655

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [5285.5535929] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [5099.33848455] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1500 1.467366717177119e+03 1.4e+00 3.43e-01  3e-01  3e-01 0:02.1
  200   3000 1.374163590037100e+03 1.4e+00 2.21e-02  2e-02  2e-02 0:04.1
  300   4500 1.373736612867955e+03 1.5e+00 1.38e-03  1e-03  1e-03 0:06.1
  400   6000 1.373735089323215e+03 1.5e+00 7.19e-05  6e-05  7e-05 0:08.1
  500   7500 1.373735086441315e+03 1.6e+00 4.50e-06  4e-06  4e-06 0:10.1
  600   9000 1.373735086429441e+03 1.6e+00 3.17e-07  2e-07  3e-07 0:12.1
  700  10500 1.373735086429368e+03 1.7e+00 2.98e-08  2e-08  2e-08 0:14.1
  741  11115 1.373735086429368e+03 1.8e+00 3.00e-08  2e-08  2e-08 0:15.0
termination on tolfun=1e-11 (Mon Apr 13 20:28:54 2020)
final/bestever f-value = 1.373735e+03 1.373735e+03
incumbent solution: [ 1.78280384 -0.85595331 -0.68104968  0.48160321 -2.20873819  0.42033632
 -1.02670656 -1.2844278  ...]
std deviations: [2.37127150e-08 2.29802455e-08 2.32260059e-08 2.42889256e-08
 2.27473676e-08 2.39518037e-08 2.34550066e-08 2.30456219e-08 ...]
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimen

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [5086.39851569] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=5086.398516, sigma=2.35e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=5086.398516, sigma=2.35e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')

  100   1500 2.343658873935195e+03 1.5e+00 4.71e-01  4e-01  5e-01 0:02.1
  200   3000 2.286753484322552e+03 1.6e+00 5.65e-02  5e-02  6e-02 0:04.2
  300   4500 2.285721955941907e+03 2.0e+00 1.08e-02  8e-03  1e-02 0:06.3
  400   6000 2.285679736436586e+03 2.6e+00 1.81e-03  1e-03  2e-03 0:08.4
  500   7500 2.285678989386924e+03 3.0e+00 2.26e-04  1e-04  2e-04 0:10.5
  600   9000 2.285678981722219e+03 3.1e+00 1.77e-05  1e-05  2e-05 0:12.6
  700  10500 2.285678981673046e+03 3.2e+00 1.43e-06  8e-07  1e-06 0:14.7
  800  12000 2.285678981672625e+03 3.4e+00 1.08e-07  6e-08  1e-07 0:16.7
  884  13260 2.285678981672625e+03 3.5e+00 3.81e-08  2e-08  4e-08 0:18.5
termination on tolfun=1e-11 (Mon Apr 13 20:29:16 2020)
final/bestever f-value = 2.285679e+03 2.285679e+03
incumbent solution: [ 5.         -4.23348337 -0.85469349  1.48802413 -3.81946356 -0.28562647
 -0.58542427 -0.89747616 ...]
std deviations: [2.34223943e-08 3.14716035e-08 3.01228831e-08 3.64355442e-08
 2.60495514e-08 3.39851254e-08 3.1135

In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F13_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F13(X_Values.iloc[i,:50])
    SVM_Fun [i] = F13(X_Values.iloc[i,50:100])
    ELN_Fun [i] = F13(X_Values.iloc[i,100:150])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([1107.56259084]), count=array([1])) 9.059227429129082e-06
SVM
ModeResult(mode=array([4403.95925843]), count=array([1])) 476.1198557366143
ELN
ModeResult(mode=array([2244.2046893]), count=array([1])) 2.436192770820441e-05
